<a href="https://colab.research.google.com/github/chetools/CHE4071_Fall2024/blob/main/DynamicDistillation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from scipy.integrate import solve_ivp
import numpy as np
from plotly.subplots import make_subplots

In [9]:
#0  Condenser
#1
#2  Nr-1
#3  Feed Stage
#4
#5
#6  Nr+Ns-1
#7  Nr+Ns:  Reboiler

Nr = 3
Ns = 4

F=1.5
z=0.4
q = 0.7

stage_holdup=0.5
feedstage_holdup = 0.75
condenser_holdup = 5.
reboiler_holdup = 5.
m_holdup=np.r_[condenser_holdup, np.full(Nr-1, stage_holdup), feedstage_holdup, np.full(Ns-1, stage_holdup) ,reboiler_holdup]
W=1.

In [ ]:
def rhs(t, v):
    R=2.
    m,x = np.split(v,2)
    dm = np.zeros_like(m)
    dmx = np.zeros_like(m)
    V = np.zeros_like(m)

    L = 0. if m<m_holdup else W*(m-m_holdup)**1.5

    #On the condenser, there is weir, the overflowing liquid is split into D and L[0]
    D = L[0]/(R+1)
    L[0]-=D

    V[1:Nr+1] = L[0] + D  #constant vapor flow rate in rectifying section
    V[Nr+1:] = V - F*(1-q)  #constant vapor flow rate in stripping section

    dm[1:-1] =  L[:-2] - L[1:-1] + V[2:] - V[1:-1]
    #  1to6      0to5      1to6     2to7     1to6
    dm[Nr]+=F
    dm[0] = V[1] - L[0] - D
    dm[-1] = L[-2] - V[-1] - L[-1]

    dmx[1:-1] =  x[:-2]*L[:-2] - x[1:-1]*L[1:-1] + y[2:]*V[2:] - y[1:-1]*V[1:-1]
